In [2]:
from pyspark.sql.functions import col
from datetime import datetime

# ============================================================
# 0. Configurações (Modo Cirúrgico - Manual)
# ============================================================
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "false")
spark.conf.set("spark.microsoft.delta.autoCompact.enabled", "false")
spark.conf.set("spark.sql.adaptive.enabled", "true")

path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_departamento_pessoal.Lakehouse/Tables/tab_gold_fato_funcionario_situacao_quantidade"

# -----------------------------
# CONSULTA
# -----------------------------
sql_query = """
WITH UltimoDiaMes AS (
    SELECT 
        CAST(DATE_TRUNC('MONTH', DATA) AS DATE) AS PrimeiroDiaMes,
        MAX(DATA) AS UltimaDataMes
    FROM tab_gold_dim_funcionario_historico
    GROUP BY DATE_TRUNC('MONTH', DATA)
)
SELECT 
    U.UltimaDataMes AS DATA,
    FH.SITUACAO,
    FH.DESCRICAO_SITUACAO,
    COUNT(*) AS QUANTIDADE_FUNCIONARIO
FROM UltimoDiaMes U
JOIN tab_gold_dim_funcionario_historico FH
    ON FH.DATA = U.UltimaDataMes
GROUP BY 
    U.UltimaDataMes,
    FH.SITUACAO,
    FH.DESCRICAO_SITUACAO
"""
df_filtrado = spark.sql(sql_query)

# ============================================================
# 2. ESTRATÉGIA FUNIL (Coalesce 1)
# ============================================================
df_otimizado = df_filtrado.coalesce(1)

# ============================================================
# 3. Contagem e Escrita
# ============================================================
# Realizamos a contagem para log (isso dispara uma ação leve no Spark)
qtd_linhas = df_otimizado.count()

# Escrevemos no disco
(
    df_otimizado.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(path_destino)
)

agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"[{agora}] Sucesso! Tabela Gold atualizada.")
print(f"Total de registros: {qtd_linhas}")

StatementMeta(, 051614f0-c741-4df2-b29a-24037d7b77a1, 4, Finished, Available, Finished)

[2025-12-05 14:16:02] Sucesso! Tabela Gold atualizada.
Total de registros: 2003
